In [3]:
!git clone https://github.com/pytorch/fairseq.git
 
%cd ./fairseq
!pip install --use-feature=in-tree-build ./

c:\Users\takah\Desktop\python_files\bokete\fairseq


fatal: destination path 'fairseq' already exists and is not an empty directory.

Usage:   
  pip install [options] <requirement specifier> [package-index-options] ...
  pip install [options] -r <requirements file> [package-index-options] ...
  pip install [options] [-e] <vcs project url> ...
  pip install [options] [-e] <local project path> ...
  pip install [options] <archive url/path> ...

option --use-feature: invalid choice: 'in-tree-build' (choose from '2020-resolver', 'fast-deps')


In [1]:
import pandas  as pd
import numpy as np
import io
from PIL import Image
import zipfile

In [2]:
train_df = pd.read_csv("../data/bokete/train.csv")
test_df = pd.read_csv("../data/bokete/test.csv")


In [3]:
# zipファイルのパス
zip_path = '../data/bokete/train.zip'

# 配列格納用のList
train_imgs = []

# zipの読み込み
with zipfile.ZipFile(zip_path, 'r') as zip_file:
    # zipファイル内の各ファイルについてループ
    for info in zip_file.infolist():
        # 「zipファイル名/」については処理をしない
        if (info.filename != 'data/'):
            # 対象の画像ファイルを開く
            with zip_file.open(info.filename) as img_file:
                # 画像のバイナリデータを読み込む
                img_bin = io.BytesIO(img_file.read())
                # バイナリデータをpillowから開く
                img = Image.open(img_bin)
                # 画像データを配列化
                img_array = np.array(img)
                # 格納用のListに追加
                train_imgs.append(img_array)

# 処理が完了後、np.arrayに変換
train_imgs = np.array(train_imgs)

C:\Users\takah\AppData\Local\Temp\ipykernel_18392\4052426647.py:25: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  train_imgs = np.array(train_imgs)


In [4]:
# zipファイルのパス
zip_path = '../data/bokete/test.zip'

# 配列格納用のList
test_imgs = []

# zipの読み込み
with zipfile.ZipFile(zip_path, 'r') as zip_file:
    # zipファイル内の各ファイルについてループ
    for info in zip_file.infolist():
        # 「zipファイル名/」については処理をしない
        if (info.filename != 'data/'):
            # 対象の画像ファイルを開く
            with zip_file.open(info.filename) as img_file:
                # 画像のバイナリデータを読み込む
                img_bin = io.BytesIO(img_file.read())
                # バイナリデータをpillowから開く
                img = Image.open(img_bin)
                # 画像データを配列化
                img_array = np.array(img)
                # 格納用のListに追加
                test_imgs.append(img_array)

# 処理が完了後、np.arrayに変換
test_imgs = np.array(test_imgs)

C:\Users\takah\AppData\Local\Temp\ipykernel_18392\726763509.py:25: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  test_imgs = np.array(test_imgs)


In [ ]:
import torch
import numpy as np
from fairseq import checkpoint_utils, distributed_utils, options, tasks, utils
from fairseq.dataclass.utils import convert_namespace_to_omegaconf
from tasks.mm_tasks.refcoco import RefcocoTask
 
from models.ofa import OFAModel
from PIL import Image
 
import cv2
import numpy
 
tasks.register_task('refcoco', RefcocoTask)
 
# turn on cuda if GPU is available
use_cuda = torch.cuda.is_available()
# use fp16 only when GPU is available
use_fp16 = False
 
# specify some options for evaluation
parser = options.get_generation_parser()
input_args = ["", "--task=refcoco", "--beam=10", "--path=checkpoints/ofa_large.pt", "--bpe-dir=utils/BPE"]
args = options.parse_args_and_arch(parser, input_args)
cfg = convert_namespace_to_omegaconf(args)